<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
        <h3>Data Generation Workflow</h3>
        <li><b>Data Extraction</b>: Extracts data from markdown files using LangChain.</li>
        <li><b>Data Generation</b>: Generates data records using sdg_hub.</li>
        <li><b>Data Uploads</b>: Uploads generated data to MinIO.</li>
        <li><b>Data Population</b>: Builds LanceDB store.</li>
</div>

In [1]:
###################################################################################################
# Imports
###################################################################################################
import boto3
import os
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import LanceDB
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.graph_vectorstores import GraphVectorStoreRetriever
from langchain_core.documents import Document
from lancedb.rerankers import LinearCombinationReranker
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import lancedb
from huggingface_hub import snapshot_download
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from transformers import AutoTokenizer
from enum import Enum
import traceback
import re
import uuid
import pandas as pd
import pprint
from datasets import Dataset
from dotenv import load_dotenv
load_dotenv()
from sdg_hub.core.flow import FlowRegistry, Flow
from flow_extensions import CustomDeleteColumnsBlock
import nest_asyncio
nest_asyncio.apply()

In [2]:
###################################################################################################
# Instance variables
###################################################################################################
endpoint_url = os.getenv('AWS_S3_ENDPOINT')
access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket = os.getenv("AWS_S3_BUCKET")
target_path = "data"

embedding_model = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2", 
    model_kwargs={"trust_remote_code":True}
)

db = lancedb.connect(f"s3://data/lancedb-dla",
    storage_options={
        "endpoint_url": endpoint_url,
        "aws_access_key_id": access_key_id,
        "aws_secret_access_key": secret_access_key,
        "s3_force_path_style": "true",
        "allow_http": "true",
    }
)

minio = boto3.client(
    's3',
    endpoint_url=endpoint_url,
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    config=boto3.session.Config(signature_version='s3v4')
)

/tmp/ipykernel_6726/2306557545.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [3]:
###################################################################################################
# Generate raw data
###################################################################################################

def build_raw_dataset(target_dir, suffix=".md", prefix=""):
    """
    Extracts STIG data from markdown files.
    """
    try:

        data, current_group_content = [], None

        files = os.listdir(target_dir)

        files = [f for f in files if f.endswith(".md") and f.startswith(prefix)]

        for file in files:
            
            filecontent = None
            
            with open(f"{target_dir}/{file}", mode="r") as f: 
                
                filecontent = f.read()
                
            headers_to_split = [("##", "Group"), ("###", "Rule")]
            
            text_splitter = MarkdownHeaderTextSplitter(headers_to_split, strip_headers=False)
            
            splits = text_splitter.split_text(filecontent)
    
            for split in splits:
    
                if "Group" in split.metadata and "Rule" not in split.metadata:
    
                    current_group_content = split.page_content
                
                if "Group" in split.metadata and "Rule" in split.metadata:
                
                    content = f"""{current_group_content.strip()}{split.page_content.strip()}"""
                    
                    data.append({"id": str(uuid.uuid4()),
                                "text": content,
                                })   

        # pprint.pprint(data)
    
        df = pd.DataFrame(data)

        ds = Dataset.from_pandas(df)
        
        return ds
        
    except Exception as e:
        print(f"Error extracting data from {target_dir}: {e}")
        
        traceback.print_exc() 

In [4]:
###################################################################################################
# Generate synthetic data
###################################################################################################
def build_synthetic_dataset(ds):
    from datasets import load_dataset, DatasetDict
    
    flow_path = "flows/rag_knowledge_generation/flow.yaml"
    
    flow = Flow.from_yaml(flow_path)
    
    flow.set_model_config(
        model=os.getenv("LLM_ID"),
        
        api_key=os.getenv('LLM_TOKEN'),
        
        api_base=os.getenv('LLM_API_BASE'),
        
        temperature=0.1,
    )
        
    dataset = flow.generate(ds, max_concurrency=100)
    
    dataset.to_json(f"{target_path}/stig_data.jsonl")

    dataset.to_parquet(f"{target_path}/stig_data.parquet")

In [5]:
###################################################################################################
# Upload data to LanceDB
###################################################################################################
# Migrate Local Search tables

def upload_to_lancedb(parquet_dir, connection):
    """
    Uploads the parquet file(s) in the provided directory to the specified lancedb database connection.
    """
    print("Migrating data to LanceDB...")
    
    try:
        for file_path in os.listdir(f"{target_path}"):
            
            if file_path.endswith(".parquet"):
        
                try:
            
                    full_path = os.path.join(target_path, file_path)
        
                    df = pd.read_parquet(full_path)

                    df['vector'] = embedding_model.embed_documents(df['log_entry'].tolist())

                    df['text_vector'] = embedding_model.embed_documents(df['text'].tolist())
        
                    table_name = file_path.split(".", 1)[0]
        
                    connection.create_table(table_name, data=df)
        
                    print(f"{table_name} migrated.")
        
                except Exception as e:
                    
                    print(f"Error uploading data from {full_path}: {e}")    
            
        print("Migration complete.")
        
    except Exception as e:
        
        print(f"Error uploading data to LanceDB: {e}")

## Run data generation pipeline
Run the pipeline!

In [6]:
os.makedirs("data", exist_ok=True)

ds = build_raw_dataset("markdown", prefix="U_RH")

build_synthetic_dataset(ds)

upload_to_lancedb(target_path, db)

print(f"Transformation complete.")

['U_RHEL_9_STIG_V1R3_Manual-xccdf.md', 'U_RH_Ansible_Automation_Controller_WS_STIG_V2R2_Manual-xccdf.md', 'U_RH_Ansible_Automation_Controller_AS_STIG_V2R2_Manual-xccdf.md', 'U_RHEL_8_STIG_V2R4_Manual-xccdf.md', 'U_RH_Ansible_Automation_Controller_WS_STIG_V1R2.md', 'U_RHEL_7_STIG_V3R15_Manual-xccdf.md', 'U_RHEL_9_STIG_V2R5_Manual-xccdf.md', 'U_RH_Ansible_Automation_Controller_AS_STIG_V1R2_Manual-xccdf.md', 'U_RH_OpenShift_Container_Platform_4-12_STIG_V2R3_Manual-xccdf.md', 'U_RH_OpenShift_Container_Platform_4-12_V1R1_Manual-xccdf.md']


[23:01:31] INFO     Loading flow from: flows/rag_knowledge_generation/flow.yaml                         ]8;id=380700;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=474921;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#140\140]8;;\

           INFO     Auto-detected 1 LLM blocks for configuration: ['generate_text_to_log_entry']        ]8;id=900832;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=39714;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#825\825]8;;\

           INFO     Successfully configured 1 LLM blocks with: model: 'openai/gpt-oss-20b', api_base:   ]8;id=157738;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=651540;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#867\867]8;;\
                    'https://openrouter.ai/api/v1', api_key:                                                       
                    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,                     
                    temperature: 0.1                                                                               

           INFO     Configured blocks: ['generate_text_to_log_entry']                                   ]8;id=733941;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=882331;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#870\870]8;;\

...

           INFO     Flow 'Log Analysis Knowledge Generation Flow for RAG' completed successfully: 1576  ]8;id=80724;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=924785;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#593\593]8;;\
                    final samples, 3 final columns                                                                 

           INFO     Converting pd.DataFrame back to datasets.Dataset to match input type                ]8;id=728038;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=849547;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#330\330]8;;\

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Migrating data to LanceDB...
stig_data migrated.
Migration complete.
Transformation complete.
